In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler

In [2]:
from sklearn.linear_model import LogisticRegression
pd.options.display.max_rows = 200

# load dataframes with timing and order information

In [3]:
wed_df = pd.read_csv('deriv/wed_df.csv',index_col=0)
wed_df.iloc[55:65]

,sub_num,wed_id,onset_recall2,offset_recall2,wed_class,wed_view_num,onset_videos,offset_videos
55,6,28,201,236,N,9,879,952
56,6,29,236,263,S,7,687,759
57,6,2,263,277,N,6,591,663
58,6,6,277,307,S,0,12,90
59,6,20,307,-1,N,2,207,279
60,7,1,0,26,N,1,111,183
61,7,34,26,71,N,10,976,1048
62,7,38,71,88,S,7,687,759
63,7,23,88,124,S,0,12,90
64,7,29,124,164,S,8,783,856


In [4]:
def load_sub_roi(sub_num,roi_name,task):
  fpath = "sub-%i_task-%s_roi-%s.npy" %(sub_num,task,roi_name)
  return np.load('data/fmri/masked/'+fpath)

In [31]:

def get_data(sub_num,roi_name,task):
  """
  task: [videos,recall2]
  returns the func data for given sub/roi/task for all 12 weddings
  """
  try:
    sub_roi_act = load_sub_roi(sub_num,roi_name,task)
    sub_wed_df = wed_df[wed_df.sub_num==sub_num]
  except:
    print('err loading sub',sub_num)
  Xact_L,ytarget_L = [],[]
  stimstr_L = []
  for idx,df_row in sub_wed_df.iterrows():
    Xact_wed = sub_roi_act[df_row.loc['onset_%s'%task]:df_row.loc['offset_%s'%task]]
    ytarget_wed = np.repeat(int(df_row.wed_class == 'N'),len(Xact_wed))
    Xact_L.append(Xact_wed)
    ytarget_L.append(ytarget_wed)
    # string identifying test sequences
    stimstr = "wed_%i-class_%s"%(df_row.wed_id,df_row.wed_class)
    stimstr_L.append(stimstr)
  return Xact_L,ytarget_L,stimstr_L




In [44]:
sub_num,roi_name,task = 32,'rglasser_PM_net','recall2'
clf_c = 1.00
## train data
Xact_train_L,ytarget_train_L,stimstr_L = get_data(sub_num,roi_name,'videos')
ytarget_train = np.concatenate(ytarget_train_L)
Xact_train = np.concatenate(Xact_train_L)
## test data
Xact_test_L,ytarget_test_L = get_data(sub_num,roi_name,'recall2')

# NORMALIZE
scaler = StandardScaler()
Xact_train = scaler.fit_transform(Xact_train)
Xact_test_L = [scaler.transform(Xact) for Xact in Xact_test_L]

# # CLASSIFIER
clf = sklearn.linear_model.LogisticRegression(solver='liblinear',C=clf_c)
clf.fit(Xact_train,ytarget_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
## EVAL LOOP: loop over 12 weddings for eval
for idx_test in range(12):
  # eval data for given wedding
  Xact_test_wed = Xact_test_L[idx_test]
  ytarget_test_wed = np.unique(ytarget_test_L[idx_test])
  # fit classifier
  yhat_wed = clf.predict_proba(Xact_test_wed)[:,ytarget_test_wed]
  print(yhat_wed.mean())


0.7634830652480973
0.48693049570784674
0.33216295137340035
0.4256327861090206
0.4310072150460813
0.28850854664181297
0.42979298462339904
0.5259397546461783
0.4240178337153569
0.37273443166362136
0.5843174402876236
0.6017817485216655


# xval loop

In [ ]:
SUB_NS = np.arange(1,45)

In [ ]:
""" 
xval loop 
"""

def xval(roi_name,clf_c):
  yhat_L = []
  for sub_num in SUB_NS:
    print(sub_num)
    try:
      sub_roi = load_sub_roi(sub_num,'videos',roi_name)
    except:
      print('failed to load sub_roi, S=',sub_num,'roi=',roi_name)
      continue
    # fold information
    for fold_num in range(6):
      fold_L_test = fold_full_L[fold_num]
      fold_L_train = get_fold_L_train(fold_num)
      print('test',fold_L_test,'train',fold_L_train)
      # TRAIN DATA
      X_TRs_train,Y_train = get_fold_info(sub_num,fold_L_train)
      X_train = sub_roi[X_TRs_train,:]
      # TEST DATA
      X_TRs_test,Y_test = get_fold_info(sub_num,fold_L_test)
      X_test = sub_roi[X_TRs_test,:]
      # NORMALIZE
      scaler = StandardScaler()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)
      # CLASSIFIER
      clf = sklearn.linear_model.LogisticRegression(solver='liblinear',C=clf_c)
      clf.fit(X_train,Y_train)
      yhat_full = clf.predict_proba(X_test)
      # split eval of test TRs into two weddings
      # and plot proba of correct schema
      yhat_first = yhat_full[:int(len(yhat_full)/2),Y_test[0]]
      yhat_second = yhat_full[int(len(yhat_full)/2):,Y_test[-1]]
      yhat_L.append(yhat_first)
      yhat_L.append(yhat_second)
  return np.array(yhat_L)


In [ ]:
## compute mean
roi_name = 'rglasser_PM_net'
clf_c = 1.0
yhat = xval(roi_name=roi_name,clf_c=clf_c)
yhat.shape

In [ ]:
M = yhat.mean(0)
S = yhat.std(0) / np.sqrt(len(yhat))

In [ ]:
## plt
ax = plt.gca()
ax.plot(M)
ax.fill_between(np.arange(len(M)),M-S,M+S,alpha=.3)
ax.axhline(0.5,c='k',ls='--')
ax.set_ylim(0.2,0.8)
ax.set_title('roi-%s_C_%.4f'%(roi_name,clf_c))
plt.savefig('figures/NvS_logreg_roi-%s_C%.4f-nsubs_%i.png'%(roi_name,clf_c,int(len(M)/12)))